In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/home/014491542/CMPE-256 Project/DataSet/finalDataset.csv',index_col=None)

In [3]:
pd.options.display.max_columns = 100

In [4]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [5]:
df.dropna(inplace=True)

In [6]:
df_new = df.drop(columns=['id','d','sales','date','wm_yr_wk','weekday','wday','is_month_start','is_month_end'])
print(df_new.shape)

(44351007, 25)


In [7]:
df_new.head()

,item_id,dept_id,store_id,cat_id,state_id,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_14,lag_21,lag_28,rolling_ave_7_7,rolling_ave_14_14,rolling_ave_21_21,rolling_ave_28_28,quarter,mday
617364,7,0,0,0,0,3,2011,0,0,0,0,0.0,0.0,0.0,0.42,0.0,0.0,0.0,0.0,2.000000,0.285714,1.952381,4.535714,1,25
617371,8,0,0,0,0,3,2011,0,0,0,0,0.0,0.0,0.0,1.77,0.0,0.0,2.0,0.0,2.000000,2.285714,1.761905,2.178571,1,25
617378,9,0,0,0,0,3,2011,0,0,0,0,0.0,0.0,0.0,3.17,0.0,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.214286,1,25
617385,11,0,0,0,0,3,2011,0,0,0,0,0.0,0.0,0.0,6.27,1.0,0.0,0.0,1.0,0.428571,0.428571,0.666667,0.678571,1,25
617392,14,0,0,0,0,3,2011,0,0,0,0,0.0,0.0,0.0,0.72,2.0,1.0,8.0,1.0,5.428571,5.571429,6.857143,4.321429,1,25


In [8]:
X = df_new
y = df['sales']
cat_features = ['item_id','dept_id','store_id','cat_id','state_id','event_name_1','event_type_1','event_name_2','event_type_2']

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=40)

In [10]:
trainData = lgb.Dataset(X_train,label=y_train,categorical_feature = cat_features,free_raw_data = False)

In [11]:
validationData = lgb.Dataset(X_test,label=y_test,categorical_feature = cat_features,free_raw_data = False)

In [21]:
params = {
    "metric":"rmse",
    "objective":"poisson",
    "force_row_wise" : True,
    "bagging_freq" : 1,
    #"sub_row":0.75,
    "verbosity": 1,
    "learning_rate":0.075,
    "num_iterations":1000,
    #"num_leaves":150,
    #"min_data_in_leaf":100,
    "lambda_l2":0.1,
}

In [22]:
history = lgb.train(params,trainData,valid_sets=[validationData],verbose_eval=50)

[50]	valid_0's rmse: 2.69388
[100]	valid_0's rmse: 2.59573
[150]	valid_0's rmse: 2.57496
[200]	valid_0's rmse: 2.54674
[250]	valid_0's rmse: 2.5295
[300]	valid_0's rmse: 2.5158
[350]	valid_0's rmse: 2.50435
[400]	valid_0's rmse: 2.49635
[450]	valid_0's rmse: 2.4865
[500]	valid_0's rmse: 2.4797
[550]	valid_0's rmse: 2.47385
[600]	valid_0's rmse: 2.46895
[650]	valid_0's rmse: 2.46385
[700]	valid_0's rmse: 2.45732
[750]	valid_0's rmse: 2.45328
[800]	valid_0's rmse: 2.44887
[850]	valid_0's rmse: 2.44631
[900]	valid_0's rmse: 2.44235
[950]	valid_0's rmse: 2.43857
[1000]	valid_0's rmse: 2.43561


In [23]:
history.save_model("/home/014491542/CMPE-256 Project/Models/LightGBM-v2.lgb")